In [561]:
import os
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
from darts import TimeSeries
from darts.metrics import mape, mae,smape, mase,rmse
from darts.dataprocessing.transformers.scaler import Scaler
from darts.models import TiDEModel

In [562]:
chunks = pd.read_csv('D:\\Azure deployment of 3UI\\alarm_raw_nov1_to_dec15.csv', chunksize=50000)
df_al = pd.concat(chunks, ignore_index=True)

In [565]:
def cleanup_alarms(df_al):
    columns_to_keep = ["alarm_time", "alarm_count", "alarm_number"]
    df_al = df_al[columns_to_keep].copy()  # Make a copy to release memory occupied by the original DataFrame
    df_al['alarm_time'] = pd.to_datetime(df_al['alarm_time'])
    df_al1 = df_al.set_index(['alarm_time'])
    df_al2 = df_al1.groupby('alarm_number')['alarm_count'].resample('15T').sum()
    df3= df_al2.reset_index()
    df_al4 = pd.pivot_table(df3.reset_index(), index='alarm_time',values ='alarm_count',columns='alarm_number',aggfunc=sum)
    df_al5 = df_al4.copy()
    df_al5.fillna(0,inplace=True)
    df_al6 = df_al5.reset_index()
    df_al6['alarm_time']=df_al6['alarm_time'].dt.tz_convert(None)
    df_al7 = df_al6.set_index(['alarm_time'])
    return df_al7


In [566]:
alarms_df=cleanup_alarms(df_al)
alarms_df

alarm_number,7,3001,3002,3003,9047,9228,9251,9252,9255,9450,...,15533452,15535435,15535732,15536090,15536091,15536732,15602891,15728650,30001151,464582601
alarm_time,,,,,,,,,,,,,,,,,,,,,
2023-11-01 00:00:00,4.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-11-01 00:15:00,4.0,0.0,0.0,0.0,6.0,7.0,0.0,0.0,3.0,0.0,...,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-11-01 00:30:00,5.0,1.0,0.0,0.0,3.0,4.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-11-01 00:45:00,5.0,1.0,0.0,0.0,9.0,8.0,0.0,0.0,5.0,0.0,...,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-11-01 01:00:00,5.0,1.0,0.0,0.0,3.0,7.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-15 22:45:00,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-12-15 23:00:00,5.0,0.0,0.0,0.0,4.0,3.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-12-15 23:15:00,4.0,0.0,0.0,0.0,6.0,5.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [567]:
df_kpi = pd.read_csv('D:\\Azure deployment of 3UI\\npm_VoLTE_kpi_nov1_to_dec15.csv')


In [568]:
df_kpi

,PERIOD_START_TIME,PLMN_Name,DN,ntasTCAT_CallSetupSuccess_Rate_MO,TCAT_CALL_AMOUNT_M20B2C2_MO,PCSCFUEOrigSIPINVRecPAT,isbcPCSCFOrigCSSRpeAccess_Type,VoLTE_Orig_Sess_Att,VoLTE_Orig_Sess_SR,VoLTE_Term_Sess_Att,VoLTE_Term_Sess_SR,BQ_INSERT_TS,ntasTCAT_CallSetupSuccess_Rate_MT,TCAT_CALL_AMOUNT_M20B2C2_MT
0,2023-12-16 00:00:00.000000 UTC,PLMN,PLMN-PLMN,99.21,96627,107859.0,95.94,108298.0,92.84,81383.0,99.90,2023-12-16 00:20:17.197942 UTC,94.55,93959
1,2023-12-15 23:45:00.000000 UTC,PLMN,PLMN-PLMN,99.34,102160,114703.0,96.05,115122.0,93.17,87811.0,99.90,2023-12-16 00:04:58.025987 UTC,94.57,100810
2,2023-12-15 23:30:00.000000 UTC,PLMN,PLMN-PLMN,99.37,111893,125083.0,96.38,125702.0,93.50,96601.0,99.91,2023-12-16 00:04:51.127308 UTC,94.45,111356
3,2023-12-15 23:15:00.000000 UTC,PLMN,PLMN-PLMN,99.37,121701,136643.0,96.39,137335.0,93.59,106769.0,99.89,2023-12-15 23:35:29.629998 UTC,94.55,122487
4,2023-12-15 23:00:00.000000 UTC,PLMN,PLMN-PLMN,99.41,136324,153168.0,96.41,154062.0,93.82,120172.0,99.92,2023-12-15 23:20:20.406029 UTC,94.86,136898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4316,2023-11-01 01:00:00.000000 UTC,PLMN,PLMN-PLMN,97.68,32078,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-04 13:52:39.255402 UTC,93.92,28573
4317,2023-11-01 00:45:00.000000 UTC,PLMN,PLMN-PLMN,98.29,35631,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-04 13:52:39.255402 UTC,94.75,31800
4318,2023-11-01 00:30:00.000000 UTC,PLMN,PLMN-PLMN,98.94,40085,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-04 13:52:39.255402 UTC,95.36,36241
4319,2023-11-01 00:15:00.000000 UTC,PLMN,PLMN-PLMN,98.69,46003,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-04 13:52:39.255402 UTC,95.32,41954


In [569]:
def clean_up_kpi(df_kpi):
    df_kpi1 = df_kpi[['PERIOD_START_TIME', 'ntasTCAT_CallSetupSuccess_Rate_MO']]
    df_kpi2 = df_kpi1.sort_index(ascending=False)
    df_kpi2['PERIOD_START_TIME']=pd.to_datetime(df_kpi2["PERIOD_START_TIME"])
    df_kpi2['PERIOD_START_TIME'] = df_kpi2['PERIOD_START_TIME'].dt.tz_convert(None)
    df_kpi2.rename(columns= {'PERIOD_START_TIME':'alarm_time'},inplace=True)
    df_kpi2.set_index('alarm_time',inplace=True)
    return df_kpi2

In [ ]:
kpi_df = clean_up_kpi(df_kpi)

In [570]:
final_df = pd.concat([kpi_df, alarms_df], axis=1)
final_df.fillna(0,inplace=True)
final_df1 = final_df.drop(final_df.index[-1])
final_df2 = pd.DataFrame(final_df1.iloc[:,:102])
final_df2.rename(columns={'ntasTCAT_CallSetupSuccess_Rate_MO':'MO'},inplace=True)
final_df3 = final_df2.reset_index()
final_df3


,alarm_time,MO,7,3001,3002,3003,9047,9228,9251,9252,...,20020,20030,20073,20075,20076,20090,20092,20093,20110,20126
0,2023-11-01 00:00:00,98.81,4.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2023-11-01 00:15:00,98.69,4.0,0.0,0.0,0.0,6.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2023-11-01 00:30:00,98.94,5.0,1.0,0.0,0.0,3.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2023-11-01 00:45:00,98.29,5.0,1.0,0.0,0.0,9.0,8.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2023-11-01 01:00:00,97.68,5.0,1.0,0.0,0.0,3.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,2023-12-15 22:45:00,99.35,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4316,2023-12-15 23:00:00,99.41,5.0,0.0,0.0,0.0,4.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4317,2023-12-15 23:15:00,99.37,4.0,0.0,0.0,0.0,6.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4318,2023-12-15 23:30:00,99.37,4.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [571]:
scaler=Scaler()
models_dict = {}


In [572]:
predict_df = pd.DataFrame()

In [573]:
final_df3

,alarm_time,MO,7,3001,3002,3003,9047,9228,9251,9252,...,20020,20030,20073,20075,20076,20090,20092,20093,20110,20126
0,2023-11-01 00:00:00,98.81,4.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2023-11-01 00:15:00,98.69,4.0,0.0,0.0,0.0,6.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2023-11-01 00:30:00,98.94,5.0,1.0,0.0,0.0,3.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2023-11-01 00:45:00,98.29,5.0,1.0,0.0,0.0,9.0,8.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2023-11-01 01:00:00,97.68,5.0,1.0,0.0,0.0,3.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,2023-12-15 22:45:00,99.35,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4316,2023-12-15 23:00:00,99.41,5.0,0.0,0.0,0.0,4.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4317,2023-12-15 23:15:00,99.37,4.0,0.0,0.0,0.0,6.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4318,2023-12-15 23:30:00,99.37,4.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [623]:
final_df4 = final_df3.set_index('alarm_time')
final_df5 = TimeSeries.from_dataframe(final_df4)
final_df5

<TimeSeries (DataArray) (alarm_time: 4320, component: 102, sample: 1)>
array([[[98.81],
        [ 4.  ],
        [ 1.  ],
        ...,
        [ 0.  ],
        [ 0.  ],
        [ 0.  ]],

       [[98.69],
        [ 4.  ],
        [ 0.  ],
        ...,
        [ 0.  ],
        [ 0.  ],
        [ 0.  ]],

       [[98.94],
        [ 5.  ],
        [ 1.  ],
        ...,
...
        ...,
        [ 0.  ],
        [ 0.  ],
        [ 0.  ]],

       [[99.37],
        [ 4.  ],
        [ 0.  ],
        ...,
        [ 0.  ],
        [ 0.  ],
        [ 0.  ]],

       [[99.34],
        [ 5.  ],
        [ 0.  ],
        ...,
        [ 0.  ],
        [ 0.  ],
        [ 0.  ]]])
Coordinates:
  * alarm_time  (alarm_time) datetime64[ns] 2023-11-01 ... 2023-12-15T23:45:00
  * component   (component) <U5 'MO' '7' '3001' ... '20093' '20110' '20126'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [625]:
scaler = Scaler()
final_df_scaled = scaler.fit_transform(final_df5)


In [629]:
target1 = final_df_scaled['MO']
cov1 = final_df_scaled.drop(['MO'],axis=1)
cov1

AttributeError: 'TimeSeries' object has no attribute 'drop'

In [574]:
final_df3.alarm_time.unique()

array(['2023-11-01T00:00:00.000000000', '2023-11-01T00:15:00.000000000',
       '2023-11-01T00:30:00.000000000', ...,
       '2023-12-15T23:15:00.000000000', '2023-12-15T23:30:00.000000000',
       '2023-12-15T23:45:00.000000000'], dtype='datetime64[ns]')

In [575]:
# Extract target variable and covariates (assuming correct column names)
target_variable = pd.DataFrame(final_df3[['alarm_time','MO']])
covariates = final_df3.drop(['MO'],axis=1)

In [576]:
target_variable

,alarm_time,MO
0,2023-11-01 00:00:00,98.81
1,2023-11-01 00:15:00,98.69
2,2023-11-01 00:30:00,98.94
3,2023-11-01 00:45:00,98.29
4,2023-11-01 01:00:00,97.68
...,...,...
4315,2023-12-15 22:45:00,99.35
4316,2023-12-15 23:00:00,99.41
4317,2023-12-15 23:15:00,99.37
4318,2023-12-15 23:30:00,99.37


In [577]:
covariates[covariates['alarm_time']=='2023-12-06 12:30:00']

,alarm_time,7,3001,3002,3003,9047,9228,9251,9252,9255,...,20020,20030,20073,20075,20076,20090,20092,20093,20110,20126
3410,2023-12-06 12:30:00,5.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [578]:
target_variable[target_variable['alarm_time']=='2023-12-06 19:30:00']

,alarm_time,MO
3438,2023-12-06 19:30:00,99.39


In [579]:
target= target_variable.set_index('alarm_time',drop=True)
covariates1= covariates.set_index('alarm_time',drop=True)

In [580]:

#Create TimeSeries objects
target_ts = TimeSeries.from_dataframe(target)  # TimeSeries for target
covariates_ts = TimeSeries.from_dataframe(covariates1)  # 
covariates_ts 

<TimeSeries (DataArray) (alarm_time: 4320, component: 101, sample: 1)>
array([[[4.],
        [1.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[4.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[5.],
        [1.],
        [0.],
        ...,
...
        ...,
        [0.],
        [0.],
        [0.]],

       [[4.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[5.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]])
Coordinates:
  * alarm_time  (alarm_time) datetime64[ns] 2023-11-01 ... 2023-12-15T23:45:00
  * component   (component) <U5 '7' '3001' '3002' ... '20093' '20110' '20126'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [229]:
# Split the TimeSeries object
#train, val = target_ts.split_before(0.8)
#cov_t , cov_v = covariates_ts.split_before(0.8)

In [581]:
print(target.index.equals(covariates1.index))


True


In [582]:
print(target.shape, target.index)
print(covariates1.shape, covariates1.index)

(4320, 1) DatetimeIndex(['2023-11-01 00:00:00', '2023-11-01 00:15:00',
               '2023-11-01 00:30:00', '2023-11-01 00:45:00',
               '2023-11-01 01:00:00', '2023-11-01 01:15:00',
               '2023-11-01 01:30:00', '2023-11-01 01:45:00',
               '2023-11-01 02:00:00', '2023-11-01 02:15:00',
               ...
               '2023-12-15 21:30:00', '2023-12-15 21:45:00',
               '2023-12-15 22:00:00', '2023-12-15 22:15:00',
               '2023-12-15 22:30:00', '2023-12-15 22:45:00',
               '2023-12-15 23:00:00', '2023-12-15 23:15:00',
               '2023-12-15 23:30:00', '2023-12-15 23:45:00'],
              dtype='datetime64[ns]', name='alarm_time', length=4320, freq=None)
(4320, 101) DatetimeIndex(['2023-11-01 00:00:00', '2023-11-01 00:15:00',
               '2023-11-01 00:30:00', '2023-11-01 00:45:00',
               '2023-11-01 01:00:00', '2023-11-01 01:15:00',
               '2023-11-01 01:30:00', '2023-11-01 01:45:00',
               '2023-11

In [583]:
covariates1.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4320 entries, 2023-11-01 00:00:00 to 2023-12-15 23:45:00
Columns: 101 entries, 7 to 20126
dtypes: float64(101)
memory usage: 3.4 MB


In [587]:
pc = covariates_ts['7'][:3455]
fc = covariates_ts['7'][3456:4320]

In [588]:
pc

<TimeSeries (DataArray) (alarm_time: 3455, component: 1, sample: 1)>
array([[[4.]],

       [[4.]],

       [[5.]],

       ...,

       [[5.]],

       [[4.]],

       [[4.]]])
Coordinates:
  * alarm_time  (alarm_time) datetime64[ns] 2023-11-01 ... 2023-12-06T23:30:00
  * component   (component) <U5 '7'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [ ]:
target

In [736]:
fc = covariates_ts[3455:4320]

In [649]:
pc = covariates_ts[:3455]
#fc = covariates_ts[3455:4320]
target_ts1 = target_ts['MO'][:3455]

# Define the TiDE model
model = TiDEModel(
    input_chunk_length=36,
    output_chunk_length=12,  
    n_epochs=15,
    temporal_width_past=4,
    num_encoder_layers=2,
    num_decoder_layers=2,
    random_state=0
)

# Train the model
model.fit(target_ts1, past_covariates=pc)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name                | Type             | Params
---------------------------------------------------------
0 | criterion           | MSELoss          | 0     
1 | train_metrics       | MetricCollection | 0     
2 | val_metrics         | MetricCollection | 0     
3 | past_cov_projection | _ResidualBlock   | 14.0 K
4 | encoders            | Sequential       | 112 K 
5 | decoders            | Sequential       | 115 K 
6 | temporal_decoder    | _ResidualBlock   | 594   
7 | lookback_skip       | Linear           | 444   
---------------------------------------------------------
242 K     Trainable params
0         Non-trainable params
242 K     Total params
0.972     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=15` reached.


TiDEModel(num_encoder_layers=2, num_decoder_layers=2, decoder_output_dim=16, hidden_size=128, temporal_width_past=4, temporal_width_future=4, temporal_decoder_hidden=32, use_layer_norm=False, dropout=0.1, use_static_covariates=True, input_chunk_length=36, output_chunk_length=12, n_epochs=15, random_state=0)

In [646]:
pred_1= model.predict(n=12)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

In [605]:
from darts.metrics import rmse, mae, mse

In [647]:
rmse_ = rmse(pred_1,target_ts['MO'][3455:])
print(f'RMSE of model is :{rmse_}')


RMSE of model is :0.42471430294918056


In [615]:
t1 = target_ts['MO'][3455:]

In [648]:
mse_ = mse(pred_1,t1)
print(f'MSE of model is :{mse_}')


MSE of model is :0.1803822391296083
